In [19]:
#| warning: false
#| code-fold: true
#import library
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from dash import dash_table
import plotly.express as px
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import geopandas as gpd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

#render ploty
pio.renderers.default = "notebook"

#import data
us_states=gpd.read_file("../final_data/tl_2020_us_state.shp")
df = pd.read_csv("../final_data/final_data.csv")
original_df = pd.read_csv("../final_data/final_data.csv")
# Initialize the Dash app
app = dash.Dash(__name__)

#normalize data
columns_to_normalize = ['Personal Income', 'Median Rent', 'Median Sale Price',
                        'Walkability', 'Unemployment', 'Purchasing Power',
                        'Violent Crime Rate', 'Non-Violent Crime Rate']

scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

# Columns where lower values are better
columns_lower_better = ['Median Rent', 'Unemployment', 'Violent Crime Rate', 'Non-Violent Crime Rate']

# Invert scaling for those columns
df[columns_lower_better] = 1 - df[columns_lower_better]

    # Define color scale mapping each political affiliation to a specific color
color_scale = {
    "Democrat": "blue",
    "Republican": "red",
    "Independent": "yellow",
    "Nonpartisan": "grey"
    }

## Introduction
The bigger the circle, the better the city is for you! Additionally, we have provided a complementary table that ranks the cities based on your preferences.

**So what are you waiting for? Find your next adventure now!**

In [27]:
app = dash.Dash(__name__)
# Define the layout of the app
app.layout = html.Div([
    html.H1("Rank by Importance",style={"color": "black"}),
    html.Div([
        html.Div([
            html.Label("Rent",style={"color": "black"}),
            dcc.Slider(
                id="Median Rent",
                min=0,
                max=1,
                step=0.1,
                value=0.5,
                marks={i/10: str(i/10) for i in range(11)},
                tooltip={"always_visible": False, "placement": "bottom"}
            ),
            html.Label("Income",style={"color": "black"}),
            dcc.Slider(
                id="Personal Income",
                min=0,
                max=1,
                step=0.1,
                value=0.5,
                marks={i/10: str(i/10) for i in range(11)},
                tooltip={"always_visible": False, "placement": "bottom"}
            ),
            html.Label("Home Sale Price",style={"color": "black"}),
            dcc.Slider(
                id="Median Sale Price",
                min=0,
                max=1,
                step=0.1,
                value=0.5,
                marks={i/10: str(i/10) for i in range(11)},
                tooltip={"always_visible": False, "placement": "bottom"}
            ),
            html.Label("Walkablity",style={"color": "black"}),
            dcc.Slider(
                id="Walkable",
                min=0,
                max=1,
                step=0.1,
                value=0.5,
                marks={i/10: str(i/10) for i in range(11)},
                tooltip={"always_visible": False, "placement": "bottom"}
            ),
            html.Label("Unemployment",style={"color": "black"}),
            dcc.Slider(
                id="Unemployment",
                min=0,
                max=1,
                step=0.1,
                value=0.5,
                marks={i/10: str(i/10) for i in range(11)},
                tooltip={"always_visible": False, "placement": "bottom"}
            ),
            html.Label("Purchasing Power",style={"color": "black"}),
            dcc.Slider(
                id="Purchasing Power",
                min=0,
                max=1,
                step=0.1,
                value=0.5,
                marks={i/10: str(i/10) for i in range(11)},
                tooltip={"always_visible": False, "placement": "bottom"}
            ),
            html.Label("Violent Crime",style={"color": "black"}),
            dcc.Slider(
                id="Crime Rate",
                min=0,
                max=1,
                step=0.1,
                value=0.5,
                marks={i/10: str(i/10) for i in range(11)},
                tooltip={"always_visible": False, "placement": "bottom"}
            ),
            html.Label("Variable to Visualize", style={"color": "black"}),
            dcc.Dropdown(
                id="variable",
                options=[{"label": i, "value": i} for i in df.columns if i not in ['Latitude', 'Longitude']],
                value=df.columns[0]
            ),
        ], style={"width": "30%", "float": "left"}),
        html.Div([
            dcc.Graph(id="city-graph")
        ], style={"width": "70%", "float": "right"}),
    ]),
])

# Define callback to update the graph based on user input
@app.callback(
    Output("city-graph", "figure"),
    [Input("Median Rent", "value"),
     Input("Personal Income", "value"),
     Input("Median Sale Price","value"),
     Input("Walkable","value"),
     Input("Unemployment","value"),
     Input("Purchasing Power","value"),
     Input("Crime Rate","value"),
     Input("variable", "value")])
def update_graph(importance_personal_income, importance_median_rent,importance_median_sale_price,importance_walkable,importance_unemployment,importance_purchasing_power,importance_crime, selected_variable):
    # Weighted sum of variables
    df["Weighted_Sum"] = (df["Median Rent"] * importance_personal_income +
                          df["Personal Income"] * importance_median_rent + df["Median Sale Price"]*importance_median_sale_price +df["Walkability"]*importance_walkable+df["Unemployment"]*importance_unemployment+
                          df["Purchasing Power"]*importance_purchasing_power+df["Violent Crime Rate"]*importance_crime)
    # Apply power transformation for better visualization
    df["Weighted_Sum"] = df["Weighted_Sum"] ** 3
    
    # Define color scale mapping each political affiliation to a specific color
    color_scale = {
        "Democrat": "blue",
        "Republican": "red",
        "Independent": "yellow",
        "Nonpartisan": "grey"
    }
    
    # Plotly scatter plot
    fig = px.scatter_geo(df, lat='Latitude', lon='Longitude', scope='usa', size="Weighted_Sum", color=selected_variable,
                     color_discrete_map=color_scale, custom_data=["Region", "Weighted_Sum"])
    
    fig.update_layout(title="Best Metro Areas by Personal Livability Score",
                      mapbox_style="carto-positron",
                      mapbox_zoom=3,
                      mapbox_center={"lat": 37.0902, "lon": -95.7129})
    
    fig.update_traces(hovertemplate='<b>%{customdata[0]}</b><br>Weighted Sum: %{customdata[1]:.2f}')
    
    return fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True,port=8052)

---------

### A closer look at each metric


In [26]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="variable",
        options=[{"label": i, "value": i} for i in original_df.columns if i not in ['Latitude', 'Longitude', 'Region', 'Political Affiliation']],
        value='Personal Income'
    ),
    # Removed dcc.Graph(id="scatter-plot"),
    dcc.Graph(id="bar-chart"),
])

@app.callback(
    Output("bar-chart", "figure"),
    [Input("variable", "value")]
)
def update_graph(variable):
    bar_chart = px.bar(original_df.sort_values(variable, ascending=False), x='Region', y=variable, color=variable)
    return bar_chart

if __name__ == "__main__":
    app.run_server(debug=True)
